In [2]:
import pandas as pd
import numpy as np
import jieba
import gensim

In [3]:
train_data=pd.read_csv("/data/linqinhong/kaggle/yitushibie/data/yitushibie/train.csv",header=None,names=["text","title"],sep="\t")
test_data=pd.read_csv("/data/linqinhong/kaggle/yitushibie/data/yitushibie/test.csv",header=None,names=["text","title"],sep="\t")

In [4]:
train_data.shape

(12100, 2)

In [5]:
train_data["process"]=train_data["text"].apply(lambda x :jieba.lcut(x))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.617 seconds.
Prefix dict has been built successfully.


In [6]:
test_data["process"]=test_data["text"].apply(lambda x :jieba.lcut(x))

In [7]:
from nltk.corpus import stopwords
stopword=stopwords.words("chinese")

In [8]:
train_data["process"]=train_data["process"].apply(lambda x : [i for i in list(x) if i not in stopword])

In [9]:
test_data["process"]=test_data["process"].apply(lambda x : [i for i in x if i not in stopword])

In [10]:
process=list(train_data["process"])+list(test_data["process"])

In [11]:
from gensim.models.word2vec import Word2Vec,Word2VecTrainables
CBOW=Word2Vec(sentences=process,vector_size=300,window=5,sg=0,epochs=30,workers=10,min_count=1)
#SkIP=Word2Vec(sentences=process,vector_size=300,window=5,sg=1,epochs=30,workers=10)

In [12]:
def get_vec(text):
    total_vec=[]
    if text==[]:
        return None
    for word in text:
        index=CBOW.wv.key_to_index[word]
        vec=np.reshape(CBOW.wv[index],(1,300))
        #vec=(CBOW.wv[index])
        total_vec.append(vec)
    return np.mean(total_vec,axis=0)

In [13]:
vocab=CBOW.wv.index_to_key

In [14]:
train_data[train_data["process"].isnull()==True]

,text,title,process


In [15]:
#train_data["process"]=train_data["process"].apply(lambda x : [i for i in list(x) if i  in vocab])

In [16]:
#test_data["process"]=test_data["process"].apply(lambda x : [i for i in list(x) if i  in vocab])

In [17]:
train_data["process"].head()

0                     [双鸭山, 淮阴, 汽车票, 13, 号]
1                                      [回家]
2            [随便, 播放, 一首, 专辑, 阁楼, 里, 佛里, 歌]
3                               [看, 墓, 王之王]
4    [想, 看, 挑战, 两把, s686, 突变, 团, 竞, 游戏, 视频]
Name: process, dtype: object

In [18]:
train_data["vec"]=train_data["process"].apply(get_vec)

In [19]:
train_data.dropna(inplace=True)

In [25]:
train_data.tail()

,text,title,process,vec
12095,一千六百五十三加三千一百六十五点六五等于几,Calendar-Query,"[一千六百五十, 三加, 三千, 一百六十五, 点, 六五, 等于]","[[-0.06534369, 0.16485937, 0.015077536, 0.2188..."
12096,稍小点客厅空调风速,HomeAppliance-Control,"[稍, 小点, 客厅, 空调, 风速]","[[-0.25253206, 0.018118115, 0.28470963, -0.185..."
12097,黎耀祥陈豪邓萃雯畲诗曼陈法拉敖嘉年杨怡马浚伟等到场出席,Radio-Listen,"[黎耀祥, 陈豪, 邓萃雯, 畲诗曼, 陈, 法拉, 敖嘉年, 杨怡, 马浚伟, 到场, 出席]","[[0.070833184, 0.11093459, -0.067549996, 0.102..."
12098,百事盖世群星星光演唱会有谁,Video-Play,"[百事, 盖世, 群星, 星光, 演唱会]","[[0.22407424, 0.2214632, -0.04750567, 0.212994..."
12099,下周一视频会议的闹钟帮我开开,Alarm-Update,"[下周一, 视频会议, 闹钟, 帮, 开开]","[[0.097743995, 0.56906253, 0.28076488, -0.0851..."


In [28]:
train_data.reset_index(drop=True,inplace=True)

In [29]:
#train_data[train_data["vec"].isnull()==True]

In [30]:
from sklearnex import patch_sklearn

In [31]:
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [32]:
labelsset=set(train_data["title"])
labelsdict={}
reverser_dict={}
for index,keys in enumerate(labelsset):
    labelsdict[keys]=index
for index,keys in enumerate(labelsset):
    reverser_dict[index]=keys

In [33]:
def convert(text):
    return labelsdict[text]

In [34]:
train_data["label"]=train_data["title"].apply(convert)

In [35]:
from sklearn.model_selection import KFold
KF=KFold(n_splits=5,shuffle=True,random_state=3407)

In [36]:
result=KF.split(train_data,train_data["label"])

In [37]:
from sklearn.svm import SVC
classifier1=SVC(kernel="linear")

In [50]:
score=[]
result=KF.split(train_data,train_data["label"])
for train_split,val_split in result:
    train1=np.concatenate(list(train_data["vec"][train_split]),axis=0)
    train2=np.array(list(train_data["label"][train_split]))
    val1=np.concatenate(list(train_data["vec"][val_split]),axis=0)
    val2=np.array(list(train_data["label"][val_split]))
    classifier1.fit(train1,train2)
    a=classifier1.score(val1,val2)
    score.append(a)

In [51]:
score

[0.893801652892562,
 0.8900826446280992,
 0.8756198347107438,
 0.8917355371900827,
 0.8970649028524184]

In [57]:
np.array([1,2])

array([1, 2])

In [365]:
train_data["vec"][train_split].shape

(9679,)

In [357]:
np.array(train_data["vec"])[[0]]

array([array([[ 3.49690905e-03,  5.33515096e-01,  3.72333638e-02,
                2.43576199e-01, -2.67117433e-02,  1.47435009e-01,
                3.92756701e-01,  1.36524081e-01, -2.11882833e-02,
                3.83401513e-01, -1.87212750e-01, -2.68994242e-01,
               -2.38685846e-01,  1.72136158e-01, -1.03461817e-01,
               -6.13948405e-01, -6.25201687e-02,  9.88147184e-02,
                2.05174953e-01, -4.20626998e-02, -7.73143992e-02,
               -1.10026598e-01, -3.46524239e-01, -1.23390080e-02,
               -9.60007310e-02, -1.94856703e-01, -4.70454127e-01,
               -8.06550235e-02, -3.58776569e-01, -2.83065140e-01,
               -2.10716322e-01, -1.68614537e-01,  2.76705652e-01,
               -2.99218237e-01, -4.95905578e-02,  9.06550437e-02,
                3.96565378e-01,  1.93912983e-01, -2.51983702e-01,
                2.38689378e-01, -3.00589651e-01,  2.28878856e-02,
               -9.63483937e-03,  1.53115064e-01,  1.63335815e-01,
          

In [360]:
get_vec(train_data["process"][0]).shape

(1, 300)

In [362]:
np.array(train_data["vec"][0]).shape

(1, 300)